# lncRNA

In [1]:
#all with evalue <1e-5
!head analyses/Myt-sxt-trinity2_blastx_sp.out

TR4_c0_g1_i1	sp|Q8R151|ZNFX1_MOUSE	53.25	77	35	1	2	229	1462	1538	1e-16	78.2
TR7_c0_g1_i1	sp|Q8VHI5|VITRN_MOUSE	40.32	62	37	0	170	355	60	121	4e-07	52.0
TR11_c0_g1_i1	sp|Q16674|MIA_HUMAN	39.60	101	47	3	36	296	10	110	3e-11	62.0
TR36_c0_g1_i1	sp|P39057|DYHC_ANTCR	72.00	75	21	0	1	225	2386	2460	1e-31	  121
TR46_c0_g1_i1	sp|Q28178|TSP1_BOVIN	55.26	76	33	1	228	1	441	515	4e-15	73.9
TR50_c0_g1_i1	sp|Q03348|PTPRA_RAT	59.46	74	28	2	6	224	262	334	1e-19	86.7
TR60_c0_g1_i1	sp|Q460N5|PAR14_HUMAN	38.24	102	62	1	623	928	797	897	2e-18	90.1
TR64_c0_g2_i1	sp|Q80UU1|ANKZ1_MOUSE	46.00	50	27	0	49	198	92	141	1e-10	60.8
TR65_c0_g1_i1	sp|O75592|MYCB2_HUMAN	54.79	73	23	2	50	241	3852	3923	5e-15	74.3
TR79_c0_g1_i1	sp|P62278|RS13_RAT	92.72	151	11	0	9	461	1	151	6e-100	  291


In [2]:
#tab file of all contigs..
!head ../data/Mytilus-SXT.trimmed.trinity.fa

>TR1_c0_g1_i1 len=249 path=[453:0-248] [-1, 453, -2]
GGCTTATCTGTCATAGTCCGCGGAAATACTTTATTTAGAATTTCAACCTGAAAACATGCG
AAACAATACTATACATGCATTATGTATCAAGTGCATAAAACAGCAGTGGGGAAACAACAA
AAGAAAAATGCATTAAACAGAATAAAAATAAATTGCATAAACAAAACTGTATACTTACTT
AAACACTAAAATAATTGAAAAAAGATAATTGAACCTGCATTGGTTGCAACGTAAACATGT
ACTTACCAG
>TR2_c0_g1_i1 len=243 path=[457:0-118 458:119-242] [-1, 457, 458, -2]
CAATAAGTCCTTACGTAAAAATTAAATTGATGCCGCGTTCCAAAATGGGACATAAAAGGC
TTCTCTTTTAGTGAATAATTTGTTGTTACAGCATTAAAAAAAAAATCACTCACTTATAAC
TATATATTTATTAAAGTATATTTCTAAGGAAAATGTTCTATCATAGCAAAATATCAGAAA


In [3]:
!cat ../data/Mytilus-SXT.trimmed.trinity.fa \
| awk '$0 ~ ">" {print c; c=0;printf substr($0,2,100) "\t"; } $0 !~ ">" {c+=length($0);} END { print c; }' \
> /Users/sr320/Desktop/test2
!head /Users/sr320/Desktop/test2


TR1_c0_g1_i1 len=249 path=[453:0-248] [-1, 453, -2]	249
TR2_c0_g1_i1 len=243 path=[457:0-118 458:119-242] [-1, 457, 458, -2]	243
TR3_c0_g1_i1 len=225 path=[203:0-224] [-1, 203, -2]	225
TR4_c0_g1_i1 len=230 path=[208:0-229] [-1, 208, -2]	230
TR5_c0_g1_i1 len=359 path=[742:0-305 743:306-329 744:330-334 745:335-358] [-1, 742, 743, 744, 745, -	359
TR5_c0_g2_i1 len=355 path=[746:0-305 747:306-329 748:330-334 749:335-354] [-1, 746, 747, 748, 749, -	355
TR6_c0_g1_i1 len=284 path=[523:0-283] [-1, 523, -2]	284
TR7_c0_g1_i1 len=444 path=[868:0-320 869:321-344 870:345-443] [-1, 868, 869, 870, -2]	444
TR8_c0_g1_i1 len=267 path=[489:0-266] [-1, 489, -2]	267


In [ ]:
!perl -e '$count=0; $len=0; while(<>) {s/\r?\n//; s/\t/ /g; if (s/^>//) { if ($. != 1) {print "\n"} s/ |$/\t/; $count++; $_ .= "\t";} else {s/ //g; $len += length($_)} print $_;} print "\n"; warn "\nConverted $count FASTA records in $. lines to tabular format\nTotal sequence length: $len\n\n";' \
analyses/Geoduck-transcriptome-v2_IDonly.fasta > analyses/Geoduck-transcriptome-v2.tab